# Preprocess

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk



In [7]:
import os
os.getcwd()

'c:\\Users\\Kevin\\Documents\\EECS_4412_Final_Project\\custom_MLP'

## Load DAtasets

In [2]:
pd.set_option('max_colwidth', 800)

TWEETS_PATH= "../data/raw_tweets_text.csv"
SENTIMENT_PATH="../data/t4sa_text_sentiment.tsv"

#load data
tweets_df = pd.read_csv(TWEETS_PATH, encoding='latin-1', header=0)
sentiment_df= pd.read_csv(SENTIMENT_PATH, sep='\t', header=0)

In [3]:
# Useful functions to help extract data from the columns

def extract_username_from_text(text):     # Extracts the username from retweets (e.g., "RT @user:").
    match = re.search(r'^RT @([^\s:]+):', text)
    return match.group(1) if match else None
     
def extract_links_from_text(text): # Extracts URLs from tweet text.
    urls = re.findall(r'https?://\S+', text)
    if not urls:
        return None
    return urls[0] if len(urls) == 1 else urls
           
def extract_hashtags_from_text(text): # Extract hashtags from the tweet text
    hashtags = re.findall(r'#\w+', text)
    if not hashtags:
        return None
    return hashtags[0] if len(hashtags) == 1 else hashtags
    
def extract_mentions_from_text(text): # Extract mentions from the tweet text
    cleaned_text = re.sub(r'^RT @[^\s:]+: ', '', text) # Remove the initial retweet username (e.g., "RT @user:")
    mentions = re.findall(r'@\w+', cleaned_text)
    if not mentions:
        return None
    return mentions[0] if len(mentions) == 1 else mentions



def clean_tweet_text(text: str):
    """
    Minimal clean text to prep for DistilBERT
    """
    # Remove "RT @user:"
    text = re.sub(r'^RT @[^\s:]+: ', '', text)

    # Remove URLs only
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
  
    
    return text.strip()

In [4]:
merged_df = pd.merge(tweets_df, sentiment_df, left_on='id', right_on='TWID')
merged_df = merged_df.drop(columns=['TWID']) # since its alr in id

main_df = merged_df.copy()
# Start adding extra columns that might help us with visualizations
# main_df['is_retweet'] = main_df['text'].str.startswith('RT ')
# main_df['username'] = main_df['text'].apply(extract_username_from_text)
# # main_df['urls'] = main_df['text'].apply(extract_links_from_text)
# main_df['hashtags'] = main_df['text'].apply(extract_hashtags_from_text)
# main_df['mentions'] = main_df['text'].apply(extract_mentions_from_text)

main_df['text'] = main_df['text'].apply(clean_tweet_text)
classes=['NEG', 'POS', 'NEU']
main_df['class'] = main_df[classes].idxmax(axis=1)
cols_to_drop = ['id', 'NEG', 'POS', 'NEU'  ]
main_df = main_df.drop(columns=cols_to_drop, axis=1)
main_df


,text,class
0,Josh Jenkins is looking forward to TAB Breeders Crown Super Sunday,POS
1,[Pic] Nichkhun from krjeong86's IG,NEU
2,Congratulations Pakistan on becoming #No1TestTeam in the world against all odds! #JI_PakZindabadRallies,POS
3,"This September, @YESmag is taking you to Maine Mendozaâs surprise thanksgiving party she threw for her fans!",POS
4,#Incredible #India #Atulya #Bharat - Land of Seekers #BeProud ð ð®ð³ :|: Plz RT,NEU
...,...,...
1179952,@LizHudston @KymWyllie @Evasmiless @meanBok @linddyloo66 @Minna1971 morning girls have a wonderful #Friday,POS
1179953,#RT #Follow Colin Kaepernick debated merits of Castro'sâ¦ - The Mercury News,NEU
1179954,I am now live on webcam find me here &gt;&gt; download our app &gt;&gt; here,NEU
1179955,Pearl Roadshow 4-piece Complete Drum Set with Cymb,NEU


In [5]:
main_df.to_csv("../project_data/dbert_ready_data.csv", index=False)